In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import itertools
import torch as th
import torch.nn as nn
from torch.optim import Adam, lr_scheduler
from torch.nn import functional as F
from tqdm import tqdm
import os

from utils import data, metrics
import Frequentist_main as FCNN
import Bayesian_main as BCNN
from Bayesian.BayesianCNN import BBBAlexNet
from Frequentist.FrequentistCNN import AlexNet

In [2]:
# Set the device
device = th.device("cuda" if th.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
# Set the parameters
priors={
    'prior_mu': 0,
    'prior_sigma': 0.1,
    'posterior_mu_initial': (0, 0.1),  # (mean, std) normal_
    'posterior_rho_initial': (-5, 0.1),  # (mean, std) normal_
}

n_epochs = 100
lr_start = [0.005, 0.001, 0.0005]
batch_size = [128, 256, 512]
hp_tuning = list(itertools.product(lr_start, batch_size))
valid_size = 0.2
beta_type = "Blundell" 

In [4]:
# Dataset and Dataloader
c10_trainset, c10_testset, c10_inputs, c10_outputs = data.getDataset('CIFAR10')
c100_trainset, c100_testset, c100_inputs, c100_outputs = data.getDataset('CIFAR100')

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# BayesianCNN with softplus on CIFAR10 hyperparametrization tuning
for lr, batch in hp_tuning:
    filename = f"results_lr{lr}_batch{batch}.csv"

    c10_train_loader, c10_valid_loader, _ = data.getDataloader(
    c10_trainset, c10_testset, valid_size, batch)
    
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Epoch', 'Training Loss', 'Training Accuracy', 'Validation Loss', 'Validation Accuracy', 'Train KL Div'])
        
        bc10_net = BBBAlexNet(c10_outputs, c10_inputs, priors, activation_type='softplus').to(device)
        bc10_criterion = metrics.ELBO(len(c10_trainset)).to(device)
        bc10_optimizer = Adam(bc10_net.parameters(), lr=lr)
        bc10_lr_sched = lr_scheduler.ReduceLROnPlateau(bc10_optimizer, patience=6, verbose=True)
        bc10_valid_loss_max = np.Inf

        ckpt_name = f"Bayesian/Models/bc10_lr{lr}_batch{batch}.pth"
        if os.path.isfile(ckpt_name):
            checkpoint = th.load(ckpt_name)
            bc10_net.load_state_dict(checkpoint['model_state_dict'])
            bc10_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            bc10_lr_sched.load_state_dict(checkpoint['scheduler_state_dict'])
            bc10_valid_loss_max = checkpoint['valid_loss_max']
            start_epoch = checkpoint['epoch'] + 1  
            print('Model loaded from {}'.format(ckpt_name))
        else:
            start_epoch = 0 

        for epoch in tqdm(range(start_epoch, n_epochs)):  # loop over the dataset multiple times
        
            bc10_train_loss, bc10_train_acc, bc10_train_kl = BCNN.train_model(bc10_net, bc10_optimizer, bc10_criterion, c10_train_loader, num_ens=1, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
            bc10_valid_loss, bc10_valid_acc = BCNN.validate_model(bc10_net, bc10_criterion, c10_valid_loader, num_ens=1, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
            bc10_lr_sched.step(bc10_valid_loss)
        
            # save model if validation accuracy has increased
            if bc10_valid_loss <= bc10_valid_loss_max:
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                    bc10_valid_loss_max, bc10_valid_loss))
                th.save({
                    'model_state_dict': bc10_net.state_dict(),
                    'optimizer_state_dict': bc10_optimizer.state_dict(),
                    'scheduler_state_dict': bc10_lr_sched.state_dict(),
                    'valid_loss_max': bc10_valid_loss,
                    'epoch': epoch
                }, ckpt_name)
                bc10_valid_loss_max = bc10_valid_loss

            writer.writerow([epoch, bc10_train_loss, bc10_train_acc, bc10_valid_loss, bc10_valid_acc, bc10_train_kl])

In [ ]:
#BayesianCNN with softplus on CIFAR100
for lr, batch in hp_tuning:
    filename = f"results_lr{lr}_batch{batch}.csv"

    c100_train_loader, c100_valid_loader, _ = data.getDataloader(
    c100_trainset, c100_testset, valid_size, batch)
    
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Epoch', 'Training Loss', 'Training Accuracy', 'Validation Loss', 'Validation Accuracy', 'Train KL Div'])
        
        bc100_net = BBBAlexNet(c100_outputs, c100_inputs, priors, activation_type='softplus').to(device)
        bc100_criterion = metrics.ELBO(len(c100_trainset)).to(device)
        bc100_optimizer = Adam(bc100_net.parameters(), lr=lr_start)
        bc100_lr_sched = lr_scheduler.ReduceLROnPlateau(bc100_optimizer, patience=6, verbose=True)
        bc100_valid_loss_max = np.Inf
        
        ckpt_name = f"Bayesian/Models/bc100_lr{lr}_batch{batch}.pth"
        if os.path.isfile(ckpt_name):
            checkpoint = th.load(ckpt_name)
            bc100_net.load_state_dict(checkpoint['model_state_dict'])
            bc100_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            bc100_lr_sched.load_state_dict(checkpoint['scheduler_state_dict'])
            bc100_valid_loss_max = checkpoint['valid_loss_max']
            start_epoch = checkpoint['epoch'] + 1  
            print('Model loaded from {}'.format(ckpt_name))
        else:
            start_epoch = 0 
        
        for epoch in tqdm(range(start_epoch, n_epochs)):  # loop over the dataset multiple times
        
            bc100_train_loss, bc100_train_acc, bc100_train_kl = BCNN.train_model(bc100_net, bc100_optimizer, bc100_criterion, c100_train_loader, num_ens=1, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
            bc100_valid_loss, bc100_valid_acc = BCNN.validate_model(bc100_net, bc100_criterion, c100_valid_loader, num_ens=1, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
            bc100_lr_sched.step(bc100_valid_loss)
        
            # save model if validation accuracy has increased
            if bc100_valid_loss <= bc100_valid_loss_max:
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                    bc100_valid_loss_max, bc100_valid_loss))
                th.save({
                    'model_state_dict': bc100_net.state_dict(),
                    'optimizer_state_dict': bc100_optimizer.state_dict(),
                    'scheduler_state_dict': bc100_lr_sched.state_dict(),
                    'valid_loss_max': bc100_valid_loss,
                    'epoch': epoch  
                }, ckpt_name)
                bc100_valid_loss_max = bc100_valid_loss

            writer.writerow([epoch, bc100_train_loss, bc100_train_acc, bc100_valid_loss, bc100_valid_acc, bc100_train_kl])

In [ ]:
# FrequentistCNN on CIFAR10
fc10_net = AlexNet(c10_outputs, c10_inputs).to(device)
fc10_criterion = nn.CrossEntropyLoss()
fc10_optimizer = Adam(fc10_net.parameters(), lr=lr_start)
fc10_lr_sched = lr_scheduler.ReduceLROnPlateau(fc10_optimizer, patience=6, verbose=True)
fc10_valid_loss_max = np.Inf

ckpt_name = 'Frequentist/Models/fc10.pth'
if os.path.isfile(ckpt_name):
    checkpoint = th.load(ckpt_name)
    fc10_net.load_state_dict(checkpoint['model_state_dict'])
    fc10_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    fc10_lr_sched.load_state_dict(checkpoint['scheduler_state_dict'])
    fc10_valid_loss_max = checkpoint['valid_loss_max']
    print('Model loaded from {}'.format(ckpt_name))

for epoch in tqdm(range(n_epochs)):

    fc10_train_loss, fc10_train_acc = FCNN.train_model(fc10_net, fc10_optimizer, fc10_criterion, c10_train_loader)
    fc10_valid_loss, fc10_valid_acc = FCNN.validate_model(fc10_net, fc10_criterion, c10_valid_loader)
    fc10_lr_sched.step(fc10_valid_loss)

    # save model if validation accuracy has increased
    if fc10_valid_loss <= fc10_valid_loss_max:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            fc10_valid_loss_max, fc10_valid_loss))
        th.save({
            'model_state_dict': fc10_net.state_dict(),
            'optimizer_state_dict': fc10_optimizer.state_dict(),
            'scheduler_state_dict': fc10_lr_sched.state_dict(),
            'valid_loss_max': fc10_valid_loss
        }, ckpt_name)
        fc10_valid_loss_max = fc10_valid_loss
            
    print('Epoch: {} \tTraining Loss: {:.4f} \tTraining Accuracy: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f}'.format(
        epoch, fc10_train_loss, fc10_train_acc, fc10_valid_loss, fc10_valid_acc))

In [ ]:
# Frequentist CNN on CIFAR100
fc100_net = AlexNet(c100_outputs, c100_inputs).to(device)
fc100_criterion = nn.CrossEntropyLoss()
fc100_optimizer = Adam(fc100_net.parameters(), lr=lr_start)
fc100_lr_sched = lr_scheduler.ReduceLROnPlateau(fc100_optimizer, patience=6, verbose=True)
fc100_valid_loss_max = np.Inf

ckpt_name = 'Frequentist/Models/fc100.pth'
if os.path.isfile(ckpt_name):
    checkpoint = th.load(ckpt_name)
    fc100_net.load_state_dict(checkpoint['model_state_dict'])
    fc100_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    fc100_lr_sched.load_state_dict(checkpoint['scheduler_state_dict'])
    fc100_valid_loss_max = checkpoint['valid_loss_max']
    print('Model loaded from {}'.format(ckpt_name))

for epoch in tqdm(range(n_epochs)):

    fc100_train_loss, fc100_train_acc = FCNN.train_model(fc100_net, fc100_optimizer, fc100_criterion, c100_train_loader)
    fc100_valid_loss, fc100_valid_acc = FCNN.validate_model(fc100_net, fc100_criterion, c100_valid_loader)
    fc100_lr_sched.step(fc100_valid_loss)

    # save model if validation accuracy has increased
    if fc100_valid_loss <= fc100_valid_loss_max:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            fc100_valid_loss_max, fc100_valid_loss))
        th.save({
            'model_state_dict': fc100_net.state_dict(),
            'optimizer_state_dict': fc100_optimizer.state_dict(),
            'scheduler_state_dict': fc100_lr_sched.state_dict(),
            'valid_loss_max': fc100_valid_loss
        }, ckpt_name)
        fc100_valid_loss_max = fc100_valid_loss
            
    print('Epoch: {} \tTraining Loss: {:.4f} \tTraining Accuracy: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f}'.format(
        epoch, fc100_train_loss, fc100_train_acc, fc100_valid_loss, fc100_valid_acc))

# After all epochs are complete, evaluate the model on the test set
fc100_test_loss, fc100_test_acc = FCNN.validate_model(fc100_net, fc100_criterion, c100_test_loader)
print('Test Loss: {:.4f} \tTest Accuracy: {:.4f}'.format(fc100_test_loss, fc100_test_acc))

In [ ]:
# Mislabel 10% of the CIFAR10 and CIFAR100 trainsets
c10_trainset_mislabel = data.mislabel_data(c10_trainset, c10_outputs, 0.1)
c100_trainset_mislabel = data.mislabel_data(c100_trainset, c100_outputs, 0.1)

# Create new dataloaders for the mislabeled datasets
c10_train_loader_mislabel, _, _ = data.getDataloader(
    c10_trainset_mislabel, c10_testset, valid_size, batch_size)
c100_train_loader_mislabel, _, _ = data.getDataloader(
    c100_trainset_mislabel, c100_testset, valid_size, batch_size)